# Multiple Regression (gradient descent)

In the first notebook we explored multiple regression using a default sklearn model. Now we will use numpy exclusively to solve for the regression weights with gradient descent.

In this notebook we will cover estimating multiple regression weights via gradient descent. You will:
* Add a constant column of 1's to the DataFrame to account for the intercept
* Export the DataFrame or its column(Series) as a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

# Load the libraries

Make sure you have the libraries listed in the course's requirement.

In [1]:
import sklearn, pandas
import numpy as np

## Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located. Sound familiar?

In [2]:
full_data = pandas.read_csv("/content/drive/MyDrive/FUNIX Progress/MLP302x_1.1-A_EN/data/kc_house_data.csv", index_col=0)

In [3]:
full_data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


If we want to do any "feature engineering" like creating new features or adjusting existing ones we modify the pandas DataFrame as seen in the previous Lab (*lab-2.ipynb*). For this notebook, however, we will work with the existing features.

## Convert to Numpy Array

In order to understand the details of the implementation of algorithms, it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 

First we need to take our organized DataFrame and extract the underlying data as a 2D numpy array (also called a matrix). To do this we can use Panda's .values property to convert the dataframe into a numpy matrix.

In [ ]:
import numpy as np # note this allows us to refer to numpy as np instead 

Now we will write a function that will accept a DataFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
* A numpy array containing the values of the output

With this in mind, complete the following function (where there's an empty line you should write a line of code that does what the comment above indicates)

In [ ]:
def get_numpy_data(data, features_title, output_title):
    if('constant' not in data):
        data['constant'] = 1 # this is how you add a constant column. Execute only when necessary
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features_title = ['constant'] + features_title # this is how you combine two lists
    # split the data into a sub-DataFrame containing your specified features (including constant).
    features_columns = data[features_title]
    # call it features_columns.

    # the following line will extract the numpy matrix from variable features_columns:
    feature_matrix = features_columns.values
    # retrieve the data associated with the output in a pandas Series
    output_column = data[output_title]
    # call it output_column

    # the following will convert the aforementioned Series into a numpy array
    output_array = output_column.values
    return(feature_matrix, output_array)

**Food for thought: The previous lab didn't specify any constant columns. Did we made a mistake there?**

For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [ ]:
(example_features, example_output) = get_numpy_data(full_data, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print(example_features[0,:]) # this accesses the first row of the data the ':' indicates 'all columns'
print(example_output[0]) # and the corresponding output.

# [   1 1180]
# 221900.0

[   1 1180]
221900.0


## Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 

$1.0*1.0 + 1.0*1180.0 = 1181.0$ 

(this is the dot product between these two arrays). If they're numpy arrays we can use `np.dot` to compute this:

In [ ]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
print(predicted_value)

1181.0


`np.dot` also works when dealing with a matrix and a vector. Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features *matrix* and the weights *vector*. With this in mind finish the following `predict_output` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [ ]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()

    return(np.dot(feature_matrix, weights))

If you want to test your code, run the following cell and observe the result:

In [ ]:
test_predictions = predict_output(example_features, my_weights)
print(test_predictions[0]) # should be 1181.0
print(test_predictions[1]) # should be 2571.0

1181.0
2571.0


# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

$(w_0 * const + w_1 *feature_1 + ... + w_i  * feature_i + ... +  w_k * feature_k - output)^2$

Where we have k features and a constant. So the derivative with respect to weight $w_i$ by the chain rule is:

$2 * (w_0 * const + w_1 *feature_1 + ... + w_i  * feature_i + ... +  w_k * feature_k - output) * feature_i$

The term inside the parenthesis is just the error (difference between prediction and output). So we can re-write this as:

$2 * error * feature_i$

That is, the derivative for the weight for feature $i$ is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for $feature_i$ is just two times the dot product between the values of $feature_i$ and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [ ]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value

    return(2*(np.dot(errors, feature)))

To test your feature derivartive run the following:

In [ ]:
(example_features, example_output) = get_numpy_data(full_data, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print(derivative)
print(-np.sum(example_output)*2) # should be the same as derivative

# -23345850016.0
# -23345850016.0

-23345850016.0
-23345850016.0


# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [ ]:
from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [ ]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        errors = predictions - output

        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])
            
            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            gradient_sum_squares = gradient_sum_squares + derivative ** 2

            # subtract the step size times the derivative from the current weight
            weights[i] = weights[i] - step_size * derivative

        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

A few things to note before we run the gradient descent. Since the gradient is a sum over all the data points and involves a product of an error and a feature the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features. 

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

# Running the Gradient Descent as Simple Regression

First let's split the data into training and test data.

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimate the parameters in the simple regression on squarefeet. The folowing cell sets up the feature_matrix, output, initial weights and step size for the first model:

In [ ]:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

Next. we run your gradient descent with the above parameters.

In [ ]:
updated_weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

How do your weights compare to those achieved in week 1 (don't expect them to be exactly the same)? 

**Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?**

In [ ]:
# You can print all the weights if you want
updated_weights

array([-46717.64952613,    283.35047387])

Use your newly estimated weights and your `predict_output` function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [ ]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

Now compute your predictions using test_simple_feature_matrix and your weights from above.

In [ ]:
# If you are a goldfish, remember: predict_output()
prediction = predict_output(test_simple_feature_matrix, updated_weights)

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 1 (round to nearest dollar)?**

In [ ]:
# the first index in a programming language is?
round(prediction[0])

358474

In [ ]:
test_output[0]

297000.0

Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [ ]:
# subtract, square, aggregate by sum, and print
sum((test_data[my_output] - prediction)**2)

267721291605091.0

# Running a multiple regression

Now we will use more than one actual feature. Use the following code to produce the weights for a second model with the following parameters:

In [ ]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

Use the above parameters to estimate the model weights. Record these values for your quiz.

In [ ]:
# Just as the previous section did
weights = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)

Use your newly estimated weights and the `predict_output` function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [ ]:
# three seconds haven't passed yet.
(test_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)
prediction = predict_output(test_feature_matrix, weights)

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?**

In [ ]:
# again, the first index
round(prediction[0])

345950

What is the actual price for the 1st house in the test data set?

In [ ]:
# find it in test_output
test_output

array([ 297000., 1578000.,  562100., ...,  369950.,  300000.,  575950.])

**Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?**

Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [ ]:
# subtract,... Wait. Copy, paste, and modify
sum((test_data[my_output] - prediction)**2)

262684098596668.03

**Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data?**

In [ ]:
262684098596668 < 267721291605091

True